In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os 
import random 
import cv2
import keras
from tensorflow.keras.layers import MaxPooling2D, Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from keras.activations import sigmoid
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras import initializers
from keras.models import Sequential

In [ ]:
#train path
train_p = "/content/drive/MyDrive/Alzheimer_s Dataset/train"
#test path
test_p = "/content/drive/MyDrive/Alzheimer_s Dataset/test"

In [ ]:
train_datagen = ImageDataGenerator(validation_split=0.1,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [ ]:
train_batches = train_datagen.flow_from_directory(directory=train_p, 
                                                  classes=['NonDemented', 'VeryMildDemented', 
                                                           'MildDemented', 'ModerateDemented'], 
                                                  target_size=(224, 224),
                                                  subset='training', 
                                                  batch_size=10)

In [ ]:
validation_batches = train_datagen.flow_from_directory(directory=train_p, 
                                                       classes=['NonDemented', 'VeryMildDemented', 
                                                                'MildDemented', 'ModerateDemented'], 
                                                       target_size=(224, 224),
                                                       subset='validation',
                                                       batch_size=10)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_batches = test_datagen.flow_from_directory(directory=test_p, 
                                                classes=['NonDemented', 'VeryMildDemented', 
                                                         'MildDemented', 'ModerateDemented'], 
                                                target_size=(224, 224),
                                                batch_size=10, 
                                                shuffle=False)

I added data augmentation to the training data generator, which includes rescaling, shearing, zooming, and horizontal flipping of the images. I also specified the target_size parameter to resize the images to a common size of 224 x 224 pixels, which is a commonly used size for image classification models.

I also separated the training data generator and the test data generator to have different parameters. For the test data generator, I only included rescaling to normalize the pixel values.

Overall, these changes should improve the performance of the machine learning model by augmenting the training data and resizing the images to a common size.

In [ ]:
class_names = ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented']
num_images = []

for cls in class_names:
    path = os.path.join(train_p, cls)
    num_images.append(len(os.listdir(path)))

In [ ]:
fig, ax = plt.subplots()
ax.bar(class_names, num_images, color=(0.7, 0.2, 0.4, 0.9))
ax.set_title('Number of Images per Class')
ax.set_xlabel('Class')
ax.set_ylabel('Number of Images')

I renamed the data dictionary to num_images to better reflect the information it stores. I also used os.path.join() to construct the path to the image directories, which is more platform-independent.

Instead of using two loops to iterate through each class and each image in the training data, I simply looped through the class_names list and used os.listdir() to count the number of images in each class directory. The resulting counts are stored in the num_images list.

Finally, I used the subplots() function to create a figure with a single subplot, and passed the ax object to the bar() function to plot the data. I also added a title, and x and y axis labels to the plot for clarity.

In [ ]:
img_size = 224
num_classes = 4
model = Sequential([
    layers.Input((img_size, img_size, 3)),
    layers.Rescaling(1./255),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Conv2D(256, 3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(300, activation='relu'),
    layers.Dense(150, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(num_classes, activation='softmax')
])


In [ ]:
metrics = [keras.metrics.CategoricalAccuracy(name='accuracy'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           keras.metrics.AUC(name='auc')]

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=metrics)

In [ ]:
epo = 5
b_size = 8

In [ ]:
history = model.fit(x=train_batches,
                    validation_data=validation_batches,
                    steps_per_epoch=len(train_batches),
                    validation_steps=len(validation_batches),
                    epochs=epo,
                    batch_size=b_size, 
                    verbose=2)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, BatchNormalization, Activation, Add
from tensorflow.keras.models import Model

def conv_block(input_tensor, filters, strides=(2, 2)):
    x = Conv2D(filters=filters, kernel_size=(3, 3), strides=strides, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    shortcut = Conv2D(filters=filters, kernel_size=(1, 1), strides=strides, padding='same')(input_tensor)
    shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def identity_block(input_tensor, filters):
    x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x

def adpm(input_shape=(224, 224, 3), classes=1000):
    input_tensor = Input(shape=input_shape)

    x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = conv_block(x, filters=64, strides=(1, 1))
    x = identity_block(x, filters=64)
    x = identity_block(x, filters=64)

    x = conv_block(x, filters=128)
    x = identity_block(x, filters=128)
    x = identity_block(x, filters=128)
    x = identity_block(x, filters=128)

    x = conv_block(x, filters=256)
    x = identity_block(x, filters=256)
    x = identity_block(x, filters=256)
    x = identity_block(x, filters=256)
    x = identity_block(x, filters=256)
    x = identity_block(x, filters=256)

    x = conv_block(x, filters=512)
    x = identity_block(x, filters=512)
    x = identity_block(x, filters=512)

    x = AveragePooling2D(pool_size=(7, 7))(x)
    x = Flatten()(x)
    x = Dense(units=classes, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=x)
    return model

In [ ]:
def (model, img_path):
    image = load_img(img_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    preds = model.predict(image)
    i = np.argmax(preds[0])
    label_to_class = {'NonDemented': 0,
                      'VeryMildDemented': 1,
                      'MildDemented': 2,
                      'ModerateDemented': 3}

    class_to_label = {v: k for k, v in label_to_class.items()}

    label = class_to_label[i]
    print(f'Predicted class: {label} | Prediction probability: {max(preds[0]) * 100}%')

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
img_path = "/content/drive/MyDrive/Alzheimer_s Dataset/test/NonDemented/26 (64).jpg"
show_heatmap(model,img_path)

In [ ]:
img_path = "/content/drive/MyDrive/Alzheimer_s Dataset/test/MildDemented/26 (19).jpg"
show_heatmap(model,img_path)